# Solve and Postprocess the Plate Problem


We will calculate deflections, strains, internal forces, stresses and utilizations for a Mindlin plate. We are also going to visualize the stresses using PyVista and save the results to disk.


In [1]:
job_id = "_00000000001"
job_data_file_path = "../outputs/" + job_id + "_data.json"

In [2]:
# Parameters
job_data_file_path = "d:\\Projects\\SigmaEpsilon\\sigmaepsilon.solid.fourier\\apps\\plate_calculator_papermill_azure_pyvista\\outputs\\_00000000001_data.json"


In [3]:
import json
from sigmaepsilon.deepdict import DeepDict, asciiprint

with open(job_data_file_path, "r") as json_file:
    job_data = json.load(json_file)
job_data : DeepDict = DeepDict.wrap(job_data)
job_data.lock()
asciiprint(job_data)

DeepDict
 +-- _meta
 +-- load
 +-- geometry
 +-- material
 +-- calculation
 |   +-- azure
 |       +-- storage
 +-- output


In [4]:
job_id = job_data["_meta","job_id"]

# geometry
length_X = job_data["geometry", "width"]
length_Y = job_data["geometry", "height"]
thickness = job_data["geometry", "thickness"]

# material properties
young_modulus = job_data["material", "youngs_modulus"]
poisson_ratio = job_data["material", "poissons_ratio"]
yield_strength = job_data["material", "yield_strength"]

# solution parameters
number_of_modes_X = job_data["calculation", "nx"]
number_of_modes_Y = job_data["calculation", "ny"]

# concentrated load
load_position = job_data["load", "position"]
load_value = job_data["load", "value"]

In [5]:
import ast
import numpy as np

from sigmaepsilon.math.linalg import ReferenceFrame

from sigmaepsilon.solid.material import MindlinPlateSection as Section
from sigmaepsilon.solid.material import (

    ElasticityTensor,
    LinearElasticMaterial,
    HuberMisesHenckyFailureCriterion_SP,
)
from sigmaepsilon.solid.material.utils import elastic_stiffness_matrix
from sigmaepsilon.solid.fourier import (
    NavierPlate,
    LoadGroup,
    PointLoad,
)

if isinstance(load_value, str):
    load_value = ast.literal_eval(load_value)
if isinstance(load_position, str):
    load_position = ast.literal_eval(load_position)
load_position = np.array(load_position)
load_value = np.array(load_value)

# set up loads
loads = LoadGroup(LC=PointLoad(load_position, load_value))

# setting up hooke's law
hooke = elastic_stiffness_matrix(E=young_modulus, NU=poisson_ratio)
frame = ReferenceFrame(dim=3)
stiffness = ElasticityTensor(hooke, frame=frame, tensorial=False)
failure_model = HuberMisesHenckyFailureCriterion_SP(yield_strength=yield_strength)
material = LinearElasticMaterial(stiffness=stiffness, failure_model=failure_model)

# section stiffness
section = Section(
    layers=[
        Section.Layer(material=material, thickness=thickness),
    ]
)
ABDS_matrix = section.elastic_stiffness_matrix()
bending_stiffness, shear_stiffness = (
    np.ascontiguousarray(ABDS_matrix[:3, :3]),
    np.ascontiguousarray(ABDS_matrix[3:, 3:]),
)

# nx, ny = (240, 320)  # for higher resolution
nx, ny = (150, 200)
x = np.linspace(0, length_X, nx)
y = np.linspace(0, length_Y, ny)
xv, yv = np.meshgrid(x, y)
evaluation_points = np.stack((xv.flatten(), yv.flatten()), axis=1)

plate = NavierPlate(
    (length_X, length_Y),
    (number_of_modes_X, number_of_modes_Y),
    D=bending_stiffness,
    S=shear_stiffness,
)

results = plate.linear_static_analysis(evaluation_points, loads)

## Calculate 2d and 3d results


In [6]:
load_case_results = results["LC"].values
strains = results["LC"].strains
z = np.linspace(-thickness / 2, thickness / 2, 20)
rng = (-thickness / 2, thickness / 2)
util, util_coords = section.utilization(
    strains=strains, rng=rng, z=z, coords=evaluation_points, return_coords=True
)
num_XY, num_Z = util_coords.shape[:2]
util_coords = util_coords.reshape((num_XY * num_Z, 3))
util = util.values.flatten()

## Save files to Parquet format

In [7]:
from sigmaepsilon.mesh import PointData

res2d_parquet_file_path = job_id + "_res2d" + ".parquet"
pd = PointData(evaluation_points, results=results["LC"].values)
pd.to_parquet(res2d_parquet_file_path)

res3d_parquet_file_path = job_id + "_res3d" + ".parquet"
pd = PointData(util_coords, util=util)
pd.to_parquet(res3d_parquet_file_path)

print("Files saved to destination!")

Files saved to destination!


In [8]:
import json, os

job_data.unlock()
absolute_file_path_res2d = os.path.abspath(res2d_parquet_file_path)
job_data["output"]["absolute_file_path_res2d"] = absolute_file_path_res2d
absolute_file_path_res3d = os.path.abspath(res3d_parquet_file_path)
job_data["output"]["absolute_file_path_res3d"] = absolute_file_path_res3d
with open(job_data_file_path, "w") as json_file:
    json.dump(job_data, json_file, indent=4)

In [9]:
job_data_file_path

'd:\\Projects\\SigmaEpsilon\\sigmaepsilon.solid.fourier\\apps\\plate_calculator_papermill_azure_pyvista\\outputs\\_00000000001_data.json'